# How to execute and import a jupyter notebook?

Include this header script in your notebooks to find more answers.

How to use inheritence in jupyter notebooks?

How to make a class out of jupter notebooks?

How to use jupyter notebooks as services?

import notebook?

TODO: not allowed to import cells that are not fully validated and unit tested.




In [1]:
// initialize
var path = require('path');
var fs = require('fs');

// provide a function for importing any notebook as a module and executing it
function getCellsOrDirectory(filename) {
    return getCells(filename, ['javascript', 'code'])
        .map((c, i) => ({
            code: c.source.join(''),
            id: path.basename(filename) + `[${i}]`,
            filename: filename,
            runInNewContext: (ctx) => {
                return runInNewContext(c.source.join(''), Object.assign(ctx || {}, {
                    __dirname: path.dirname(filename),
                    __filename: filename,
                }), {filename: path.join(
                    path.dirname(filename),
                    path.basename(filename) + `[${i}]`)}, false);
            }
        }))
}

// How to test if a notebook has already been imported
function importNotebook(notebook, ctx = {}) {
    if (typeof notebook === 'undefined') {
        return Promise.resolve({});
    }
    var cells;
    // TODO: do Core notebooks synchronously?
    if(typeof notebook === 'string' && fs.existsSync(path.resolve(__dirname, notebook))) {
        cells = getCellsOrDirectory(path.resolve(path.resolve(__dirname, notebook)));
    } else {
        cells = interpret(notebook);
        if( typeof cells.code !== 'undefined') {
            return cells.runInNewContext(ctx);
        }
    }
    console.log(`importing ${notebook} - ${cells.length} cells`)
    return cells.reduce((obj, c, i) => {
        var result = c.runInNewContext(Object.assign(ctx, obj));
        if(typeof result === 'object'
           && typeof result[Object.keys(result)[0]] === 'function') {
            const func = result[Object.keys(result)[0]];
            obj[cells[i].id] = func;
            obj[func.name] = obj[cells[i].id];
        }
        if (typeof result === 'function') {
            obj[cells[i].id] = result;
            obj[result.name] = obj[cells[i].id];
        }
        obj[i] = result;
        Object.assign(ctx, obj);
        return obj;
    }, {});
}

module.exports.import = importNotebook;

// $$.done(); for our dumb parser



How to stream json using a {match} function?


In [ ]:
// initialize


// TODO: replace json path with this
function walkJson() {
    
}

function selectJson() {
    
}

var fs = require('fs');
var {JSONPath} = require('jsonpath-plus');
var fileCache = {};

function streamJson(file, path, match = ((c) => true)) {
    const json = JSON.parse(fs.readFileSync(file).toString());
    const nodes = JSONPath({json: Object.assign({}, json), path: '$..', resultType: 'all', callback: n => {
        if(typeof n.value !== 'object' || n.value === null) {
            return;
        }
        try {
            return match({
                path: n.path.split(/[\\'\]\[\$]+/igm).slice(1),
                value: n.value
            });
        } catch (e) {
            console.error(e);
            return;
        }
    }});
}

module.exports.streamJson = streamJson;

// $$.done() for our dumb parser


How to convert regexp matches to an array?

In [ ]:
// initialize

function regexToArray(ex, str, i = 0) {
    var co = []; var m;
    while ((m = ex.exec(str)) && co.push(m[i])) ;
    return co;
};
module.exports.regexToArray = regexToArray;

// $$.done() for our dumb parser



How to parse cells from a notebook?



In [ ]:
// initialize

function getCells(notebook, types = ['*', 'code']) {
    //console.info('reading notebook ' + notebook);
    var kernel = {};
    var cells = [];
    streamJson(notebook, [true, {emitPath: true}], (match) => {
        if (match.path[0] === 'metadata'
            && match.path[1] === 'kernelspec') {
            kernel = match.value;
        } else if (match.path[0] === 'cells'
            && types.indexOf(match.value.cell_type) > -1) {
            cells[cells.length] = match.value;
        }
    });
    if (types.indexOf(kernel.language) === -1
        && types.indexOf('*') === -1) {
        console.info('tried to import ' + types + JSON.stringify(kernel));
        return [];
    }
    return cells.map(c => Object.assign(c, {language: kernel.language}));
}

module.exports.getCells = getCells;


// $$.done() for our dumb parser


How to run all promises sequentially?



In [ ]:
// initialize

function runAllPromises(promises) {
    return promises.reduce((promise, func) => {
        return promise.then(result => {
            return (typeof func == 'function'
                ? (new Promise(func)) : Promise.resolve(func))
                .then(Array.prototype.concat.bind(result));
        });
    }, Promise.resolve([]));
}

module.exports.runAllPromises = runAllPromises;

// $$.done() for our dumb parser



How to turn jupyter notebooks in to modules?

# interpret(query)

This function has 4 modes:

```
search terms
search notebook.ipynb
search notebook.ipynb[0]
search notebook.ipynb[search terms]
```

Passing in a string returns the top match of a single cell, unless the string names a notebook in which case it returns and Array of all the cells in the notebook.
`[0]` returns the index of code cells, markdown cells are accumulated leading up to a code cell.
`[search terms]` searches the markdown within the specified notebook, it's garunteed to return a cell from the notebook or no match error.



In [1]:
// initialize

// TODO: remove runInContext stuff and purge with fuseSearch

var path = require('path');
var first = true;

function interpret(query) {
    if(first) {
        first = false;
        cacheAll(path.join(__dirname, '../'));
    }
    if(typeof query === 'undefined' || query === null || query === false) {
        // special lookup function for retrieving the entire index
        return cacheCells([], '');
    }
    const searches = (typeof query === 'string'
                      ? [query]
                      : query).map(q => fuseSearch(q));
    const fileMode = typeof query === 'string'
        && query.indexOf('.ipynb') > -1
        && query.indexOf('[') === -1;
    // TODO: move this logic for creating the context in to the question
    //  interpreter so it don't have to loop so many times?
    const results = (fileMode ? searches[0] : searches).map(r => {
        if (typeof r === 'undefined') {
            return r;
        }
        // TODO: insert niceName and getExports here?
        // must have a unique id for each unique cell so that
        // individual cells can serve as modules as well as notebooks
        // adding the cell id as a part of the filename
        var question = r.questions && r.questions[0]
            ? (' aka ' + r.questions[0].substr(0, 50))
            : '';
        r.runInNewContext = (ctx) => {
            return runInNewContext(
                r.code,
                Object.assign({
                    __filename: r.filename
                }, ctx || {}),
                {filename: path.join(path.dirname(path.resolve(r.filename)), r.id) + question}, false);
        };
        return r;
    });
    return typeof query === 'string' && !fileMode
        ? results[0]
        : results;
}

module.exports.interpret = interpret;

// $$.done() for our dumb parser



[Function: interpret]

Import arbitrary code in to the current context?

Import arbitrary code in to a new context?

How does node module require work?

# runInNewContext(code, context)

Imports arbitrary code as a Node module.

## options

* useCache: true
* __filename: path or filename in errors
* 


TODO: webpack other languages and import?

TODO: use zeromq to import any language using other kernels?

TODO: use language server from VS Code?



In [ ]:
// initialize

var Module = require('module').Module;
var path = require('path');

function requireCode(code, pathToCode, tmpGlobal) {

    // must have a new name for every generation otherwise cache will be returned
    var filepath = path.resolve(process.cwd(), pathToCode);
    var mtime = fs.statSync(filepath
    // TODO: find a better way to serve notebook names
        .replace(/\.ipynb(\[[0-9]+\]).*$/ig, '.ipynb')).mtime.getTime();
    var dirname = path.dirname(filepath);

// TODO: don't use cache of parent modules that have includes that have changed
    var cachedModule = Module._cache[filepath];
    if (cachedModule && (mtime || 0) < (cachedModule.buildTime || 0)) {
        tmpGlobal.module = cachedModule;
        return cachedModule.exports;
    }

    var mod = new Module(filepath, tmpGlobal.module);
    tmpGlobal.module = mod;
    tmpGlobal.exports = mod.exports = {};
    Module._cache[filepath] = mod;
    mod.parent = module;
    mod.buildTime = (new Date()).getTime();
    mod.filename = filepath;
    mod.paths = Module._nodeModulePaths(dirname);
    
    // TODO: this is basically the magic of the 'rewire' module, reuse this?
    const validVars = Object.keys(tmpGlobal)
        .filter(k => k.match(/^[a-z_][a-z0-9_]*$/i))
        .concat(['$$'])
        .join(',');

    console.info('compiling ' + pathToCode);
    
    mod._compile(`module.exports = (({${validVars}}) => {
${code}
})`, filepath);
    // assign the actual imports internally by calling the func export
    var result = mod.exports;
    tmpGlobal.exports = tmpGlobal.module.exports = {};
    result(tmpGlobal);
    mod.loaded = true;
    return mod.exports || result;
}

// TODO: remove options
function runInNewContext(code, ctx, options) {
    const oldDir = process.cwd();
    const tmpGlobal = ctx || {};
    Object.assign(tmpGlobal, ctx);
    Object.assign(options || {}, { lineOffset: -1 });
    options.filename = options.filename || ctx.__filename;
    // TODO: use transpiler to replace Core imports all together, not just this context
    tmpGlobal.importer = require('../Core');
    // remove reference to Core and inject this context
    //code = code.replace(/.*importer.*require\('\.\.\/Core'\).*;*\s*/igm, '');
    const result = requireCode(code, options.filename, tmpGlobal);
    return tmpGlobal.module.exports || result;
}

module.exports.runInNewContext = runInNewContext;

// $$.done() for our dumb parser


This is a complex question to answer.  Perhaps using IBM Watson language alchemy?

How to find functions in jupyter notebooks?

How to cache notebook questions in fuse?



In [ ]:
// initialize

var path = require('path');
var MATCH_QUESTIONS = (/^.*\?.*$/igm);

// store code markdown results
if (typeof cellCache === 'undefined') {
    var cellCache = [];
}

function clearNotebook(filename) {
    // update cells when files change, remove old cells from cache
    cellCache.forEach(c => {
        if(c.filename === filename) {
            cellCache.splice(cellCache.indexOf(c), 1);
        }
    });
}

function getQuestions(source, markdown) {
    var questions = regexToArray(MATCH_QUESTIONS, markdown)
        .map(r => r.replace(/how to|\?/ig, '').trim())
        .concat(regexToArray(MATCH_QUESTIONS, source)
            .filter(r => r.match(/how to/ig))
            .map(r => r.replace(/how to|\?/ig, '').trim()))
        // find the shortest words from the query to match the same cell
    questions.sort((a, b) => a.length - b.length)
    return questions;
}

// TODO: convert to pattern like group everything leading up to match
function accumulateMarkdown(cells, filename) {
    // read markdown leading up to code cells
    var codes = cells.filter(c => c.cell_type === 'code')
    return codes
        .map((c, i) => Object.assign(c, {
            from: i > 0 ? (cells.indexOf(codes[i-1]) + 1) : 0,
            to: cells.indexOf(c)
        }))
        .map((c, i) => Object.assign(c, {
            markdown: cells.slice(c.from, c.to).map(m => m.source.join('')),
            code: c.source.join('')
        }))
}

function cacheCells(cells, filename) {
    if(!filename) {
        return cellCache;
    }
    
    clearNotebook(filename);
    var newCache = accumulateMarkdown(cells);
    newCache.forEach((c, i) => {
        var questions = getQuestions(c.code, c.markdown);
        if(questions.length === 0) questions = [''];
        questions.forEach(q => cellCache.push(Object.assign({}, c, {
            id: path.basename(filename) + '[' + i + ']',
            filename: filename,
            mtime: (new Date()).getTime(),
            notebook: path.basename(filename),
            questions: questions,
            question: q,
        })));
    });
    return cellCache;
}

module.exports.cacheCells = cacheCells;

// $$.done() for our dumb parser


How to display interpreted results in markdown?



In [ ]:
// initialize

function resultMarkdown(res) {
    return ('\n\n\n' + res.length + ' match'
        + (res.length !== 1 ? 'es' : '')
        + ' found: ' + res.join(' , ') + '\n\n\n'
        + (res.length > 0
            ? ('\n\n\n' + res.markdown.join('\n') + '\n\n\n'
                + '```\n\n\n' + res.code + '\n\n\n```\n\n\n')
            : ''));
};

function interpretMarkdown(results) {
    return (typeof results[0] !== 'undefined' && typeof results[0] !== 'string'
        ? results.reduce((str, res) => {
            str += resultMarkdown(res);
            return str;
        }, '')
        : resultMarkdown(results));
}

module.exports.interpretMarkdown = interpretMarkdown;

// $$.done() for our dumb parser


How to search notebook questions?

How to answer questions using jupyter notebooks and simple search?

How to search jupter notebooks for questions and answers?


In [ ]:
// initialize

var path = require('path');
var fs = require('fs');
var Fuse = require('fuse.js');

var FUSE_CONFIG = {
    caseSensitive: false,
    findAllMatches: true,
    distance: 100,
    threshold: 0.8,
    tokenize: true,
    shouldSort: true,
    keys: ['question'],
    id: 'id'
}
var token = new Fuse(cacheCells([], ''), FUSE_CONFIG);
var fuse = new Fuse(cacheCells([], ''), Object.assign({}, FUSE_CONFIG, {
    tokenize: false
}));

// filter query results by filename
var filterFilename = (arr, fname) => arr
    .filter(id => id.substr(0, fname.length + 1) === fname + '[');

function doSearch(search) {
    var tokenResults = token.search(search);
    var fuseResults = fuse.search(search);
    return fuseResults
        .filter(s => tokenResults.includes(s))
        .concat(tokenResults.filter(s => fuseResults.includes(s)))
}

function getAllCells() {
    return cacheCells([], '');
}

function fuseSearch(queries) {
    const cellIds = getAllCells().map(c => c.id)
        .filter((i, k, arr) => arr.indexOf(i) === k);
    const query = path.basename(queries)
        .split(/[\[\]]/ig);
    const fileMode = query.length === 1
        && query[0].indexOf('.ipynb') > -1;
    let searchResults;
    if (query.length === 3) {
        var isNumeric = parseInt(query[1]) + '' === query[1];
        searchResults = isNumeric
            ? [getAllCells().filter(cell => cell.id
                                    === path.basename(queries))[0].id]
            : filterFilename(doSearch(query[1]), query[0])
    } else if (fileMode) {
        searchResults = filterFilename(cellIds, path.basename(queries))
            .sort((a, b) => parseInt(
            a.split(/[\[\]]/ig)[1]) - parseInt(b.split(/[\[\]]/ig)[1]));
    } else {
        searchResults = doSearch(queries);
    }
    if(searchResults.length === 0 || typeof searchResults[0] === 'undefined') {
        throw new Error(`Nothing found for ${JSON.stringify(queries)
                        .substr(0, 200)}`);
    }
    return fileMode
        ? searchResults.map(r => interpretObject(r))
        : interpretObject(searchResults[0]);
}

function interpretObject(cacheId, allCells) {
    var cache = getAllCells().filter(cell => cell.id === cacheId)[0];
    try {
        // TODO: this doesn't work because every cell
        //   in the tree would need to be invalidated
        const mtime = fs.statSync(cache.filename).mtime.getTime();
        if(mtime > (cache.mtime || 0)) {
            cacheCells(getCells(cache.filename, ['*', 'markdown', 'code']),
                       cache.filename);
            return getAllCells().filter(cell => cell.id === cacheId)[0];
        }
    } finally {
        return cache;
    }
}

module.exports.fuseSearch = fuseSearch;

// $$.done() for our dumb parser


find all notebooks?

How to get markdown from all notebooks in {directory}?

How to interpret a jupyter {directory}?



In [ ]:
// initialize

var path = require('path');
var glob = require('glob');
var fs = require('fs');
var assert = require('assert');

var listInProject;
function cacheAll(dirname) {
    if(typeof listInProject === 'undefined') {
        // cache the filesystem notebook so we can use it in loading
        const filesystemPath = path.join(__dirname, './files.ipynb');
        assert(fs.existsSync(filesystemPath), 'missing ' + path.resolve(filesystemPath));
        cacheCells(getCells(filesystemPath, ['*', 'markdown', 'code']), filesystemPath);
        listInProject = importNotebook('list project files');
        assert(!!listInProject, 'something is seriously wrong with indexing notebooks')
    }
    return listInProject(dirname, '{,*,*/,*/*/*,*/*/*/*}*.ipynb')
        .map(n => cacheCells(getCells(n, ['*', 'markdown', 'code']), n))
}

module.exports.cacheAll = cacheAll;

// $$.done() for our dumb parser
